In [2]:
# Установка модулей
!pip install -q -U openai

In [1]:
# Импорт библиотек
import openai

import pandas as pd
import numpy as np

from tqdm import tqdm

In [2]:
# Креды для подключения
openai.api_key = "sk-or-vv-837a4639a1aed7f87b3b07754f11eaad6f97384ed24ae1bb4ce3759611160519" 
openai.base_url = "https://api.vsegpt.ru/v1/"

In [3]:
# Проверка работоспособности запросов
prompt = \
"""
Напиши код на Python, который бы вывел среднее в датафрейме, где колонка "sales"
"""
 
messages = []
messages.append({"role": "user", "content": prompt})

response_big = openai.chat.completions.create(
    model="openai/gpt-4-1106-preview",
    messages=messages,
)

response = response_big.choices[0].message.content
print(prompt)
print()
print(response)


Напиши код на Python, который бы вывел среднее в датафрейме, где колонка "sales"


Чтобы вывести среднее значение в колонке "sales" в датафрейме при помощи библиотеки Pandas, нужно сначала убедиться, что данные в этой колонке являются численными. После этого можно использовать метод `.mean()` для вычисления среднего. Вот пример кода на Python:

```python
import pandas as pd

# Создаем пример датафрейма
data = {
    'sales': [100, 200, 300, 400, 500],
    'other_column': [10, 20, 30, 40, 50]
}

# Создаем датафрейм
df = pd.DataFrame(data)

# Вычисляем среднее значение по колонке 'sales'
average_sales = df['sales'].mean()

# Выводим результат
print(f"Среднее значение продаж: {average_sales}")
```

Если требуется получить средние значения по всем числовым колонкам в датафрейме, то можно использовать метод `.mean()` для всего датафрейма:

```python
average_values = df.mean()
print(average_values)
```

Обратите внимание, что метод `.mean()` по умолчанию игнорирует нечисловые значения.



In [85]:
# Импортируем обработанный в BIO-нотации файл
result_df = pd.read_csv("data_bio.csv", encoding="utf-16")
result_df

,id,token,tag
0,16123526,обязанности,O
1,16123526,:,O
2,16123526,перевод,O
3,16123526,технической,O
4,16123526,документации,O
...,...,...,...
1985222,37498527,10:00-18:00,O
1985223,37498527,лояльное,O
1985224,37498527,и,O
1985225,37498527,грамотное,O


In [98]:
random_id = result_df.sample(1, random_state=100)["id"].values[0]

input_example = ' '.join(list(result_df[result_df["id"] == random_id]["token"]))
input_example

'обязанности : охрана общественного порядка и общественной безопасности требования : -полное среднее образование , -служба в вс ( для военнообязанных ) условия : полный соцпакет ,'

In [99]:
# Проверка работоспособности модели
prompt = f"""
Ты - алгоритм, который специализируется на извлечении навыков из текстов вакансий. Ниже приведен текст вакансии. Твоя задача - выделить перечень ключевых навыков, которые требуются для этой работы. Навыки должны быть представлены в виде списка, разделенного запятыми и в том же порядке, что и в исходном тексте.

Пример 1 текста вакансии и ожидаемый ответ от модели:
Вход: 'Обязанности: приём ВХОДЯЩИХ звонков со всей России; консультация клиентов по вопросам тематики проекта. Требования: уверенный пользователь ПК; грамотная речь. Условия: время обучения нами оплачивается; оплата фиксированная за количество отработанных часов; выплата заработной платы 2 раза в месяц; по собственному желанию, оператор может брать доп. часы на подработку; бонусная система - премия; оформление официальное или ГПХ, возможно совмещение.'
Вывод: 'прием входящий звонков, консультация клиентов, уверенный пользователь ПК, грамотная речь'

Пример 2 текста вакансии и ожидаемый ответ от модели:
Вход: 'Обязанности:  Печать пакета документов для водителя (маршрутный лист, накладные, счета-фактуры), приём водителей — обработка «вычерков» в накладных, получение денежных средств от водителей с последующей передачей в кассу. Возможно участие в приёме заказов по телефону от торговых представителей.  Требования:  Базовые знание компьютера и 1С, знание Excel — приветствуется. Коммуникабельность, обучаемость и стрессоустойчивость. Наличие опыта аналогичной работы станет преимуществом, отсутствие — не является преградой.    Условия:  График работы посменный 2 через 2 дня, рабочая смена — 12 часов (с 14:00 до 2:00, возможны другие режимы — обсуждается на собеседовании). Место работы МО Дмитровский р-н с. Орудьево ул.Фабричная'
Вывод: 'Печать пакета документов, приём водителей, обработка «вычерков» в накладных, получение денежных средств от водителей, базовые знание компьютера и 1С, знание excel, коммуникабельность, обучаемость, стрессоустойчивость, наличие опыта аналогичной работы'

Теперь также проанализируй следующий текст вакансии и выяви ключевые навыки, как на примере:
Вход: '{input_example}'
Вывод:
"""

messages = []
messages.append({"role": "user", "content": prompt})

response_big = openai.chat.completions.create(
    model="gusev/saiga-mistral-7b",
    messages=messages,
    temperature=0.1,
    n=1
)

response = response_big.choices[0].message.content
print(response)

охрана общественного порядка, общественной безопасности, полное среднее образование, служба в ВС (для военнообязанных)


In [100]:
skills_found = list(map(lambda x: x.split(), response.split(', ')))
tag_model = []

skill_idx = 0
token_idx = 0
for row in result_df[result_df["id"] == random_id].values:
    if row[1] == skills_found[skill_idx][token_idx]:
        if token_idx == 0:
            tag_model.append('B-SKILL')
            if token_idx + 1 < len(skills_found[skill_idx]):
                token_idx += 1
            else:
                skill_idx += 1
                token_idx = 0
        else:
            tag_model.append('I-SKILL')
            if token_idx + 1 < len(skills_found[skill_idx]):
                token_idx += 1
            else:
                skill_idx += 1
                token_idx = 0
    else:
        tag_model.append('O')

In [104]:
def calculate_metrics(predictions, true_tags, skill_tag_indices):
    """
    Расчет precision, recall и F1-score только для тегов B-SKILL и I-SKILL.
    
    predictions: Предсказания модели (после применения torch.max).
    true_tags: Истинные теги.
    skill_tag_indices: Индексы тегов B-SKILL и I-SKILL.
    """
    # Инициализация счетчиков для TP, FP, FN
    tp = 0
    fp = 0
    fn = 0

    # Преобразование векторов для удобства сравнения
    pred_flat = predictions
    true_flat = true_tags
    
    for skill_tag in skill_tag_indices:
        # True Positives: Предсказанный тег совпадает с истинным
        tp += ((pred_flat == skill_tag) & (true_flat == skill_tag)).sum().item()
        # False Positives: Предсказан тег, но истинный тег другой
        fp += ((pred_flat == skill_tag) & (true_flat != skill_tag)).sum().item()
        # False Negatives: Тег не был предсказан, хотя он есть
        fn += ((pred_flat != skill_tag) & (true_flat == skill_tag)).sum().item()

    # Расчет метрик
    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return precision, recall, f1

In [105]:
calculate_metrics(np.array(tag_model), np.array(result_df[result_df["id"] == random_id]["tag"]), ['B-SKILL', 'I-SKILL'])

(0.2, 1.0, 0.33333333333333337)

In [109]:
precision_lst = []
recall_lst = []
f1_lst = []

# Теперь через цикл пройдемся по нескольким вакансиям и проверим модель
random_ids = result_df.sample(30, random_state=100)["id"].values

for random_id in tqdm(random_ids[:]):

    input = ' '.join(list(result_df[result_df["id"] == random_id]["token"]))
    
    # Проверка работоспособности модели
    prompt = f"""
    Ты - алгоритм, который специализируется на извлечении навыков из текстов вакансий. Ниже приведен текст вакансии. Твоя задача - выделить перечень ключевых навыков, которые требуются для этой работы. Навыки должны быть представлены в виде списка, разделенного запятыми и в том же порядке, что и в исходном тексте.

    Пример 1 текста вакансии и ожидаемый ответ от модели:
    Вход: 'Обязанности: приём ВХОДЯЩИХ звонков со всей России; консультация клиентов по вопросам тематики проекта. Требования: уверенный пользователь ПК; грамотная речь. Условия: время обучения нами оплачивается; оплата фиксированная за количество отработанных часов; выплата заработной платы 2 раза в месяц; по собственному желанию, оператор может брать доп. часы на подработку; бонусная система - премия; оформление официальное или ГПХ, возможно совмещение.'
    Вывод: 'прием входящий звонков, консультация клиентов, уверенный пользователь ПК, грамотная речь'

    Пример 2 текста вакансии и ожидаемый ответ от модели:
    Вход: 'Обязанности:  Печать пакета документов для водителя (маршрутный лист, накладные, счета-фактуры), приём водителей — обработка «вычерков» в накладных, получение денежных средств от водителей с последующей передачей в кассу. Возможно участие в приёме заказов по телефону от торговых представителей.  Требования:  Базовые знание компьютера и 1С, знание Excel — приветствуется. Коммуникабельность, обучаемость и стрессоустойчивость. Наличие опыта аналогичной работы станет преимуществом, отсутствие — не является преградой.    Условия:  График работы посменный 2 через 2 дня, рабочая смена — 12 часов (с 14:00 до 2:00, возможны другие режимы — обсуждается на собеседовании). Место работы МО Дмитровский р-н с. Орудьево ул.Фабричная'
    Вывод: 'Печать пакета документов, приём водителей, обработка «вычерков» в накладных, получение денежных средств от водителей, базовые знание компьютера и 1С, знание excel, коммуникабельность, обучаемость, стрессоустойчивость, наличие опыта аналогичной работы'

    Теперь также проанализируй следующий текст вакансии и выяви ключевые навыки, как на примере:
    Вход: '{input}'
    Вывод:
    """

    messages = []
    messages.append({"role": "user", "content": prompt})

    response_big = openai.chat.completions.create(
        model="gusev/saiga-mistral-7b",
        messages=messages,
        temperature=0.1,
        n=1
    )

    response = response_big.choices[0].message.content
    
    skills_found = list(map(lambda x: x.split(), response.split(', ')))
    tag_model = []

    skill_idx = 0
    token_idx = 0
    for row in result_df[result_df["id"] == random_id].values:
        if row[1] == skills_found[skill_idx][token_idx]:
            if token_idx == 0:
                tag_model.append('B-SKILL')
                if token_idx + 1 < len(skills_found[skill_idx]):
                    token_idx += 1
                else:
                    skill_idx += 1
                    token_idx = 0
            else:
                tag_model.append('I-SKILL')
                if token_idx + 1 < len(skills_found[skill_idx]):
                    token_idx += 1
                else:
                    skill_idx += 1
                    token_idx = 0
        else:
            tag_model.append('O')
    
    metrics = calculate_metrics(np.array(tag_model), np.array(result_df[result_df["id"] == random_id]["tag"]), ['B-SKILL', 'I-SKILL'])

    precision = metrics[0]
    recall = metrics[1]
    f1 = metrics[2]

    precision_lst.append(precision)
    recall_lst.append(recall)
    f1_lst.append(f1)

print("Средний precision:", sum(precision_lst) / len(precision_lst))
print("Средний recall:", sum(recall_lst) / len(recall_lst))
print("Средняя f-мера:", sum(f1_lst) / len(f1_lst))

  3%|▎         | 1/30 [20:57<10:07:42, 1257.32s/it]


InternalServerError: Internal Server Error